In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#한글폰트 가져오기
from matplotlib import rc
rc('font', family='NanumGothic')

# Warning 메세지를 뜨지 않게 해줌
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder, StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean
from sklearn.metrics import euclidean_distances



### 데이터 불러오기

In [4]:
df = pd.read_csv("C:/Users/jeong/Downloads/국민건강보험공단_건강검진정보_20231231/국민건강보험공단_건강검진정보_2023.CSV", encoding='cp949')
df_origin = df.copy()

In [5]:
pd.set_option('display.max_columns', None)
display(df.head())

,기준년도,가입자일련번호,시도코드,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),총콜레스테롤,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,구강검진수검여부,치아우식증유무,결손치 유무,치아마모증유무,제3대구치(사랑니) 이상,치석
0,2023,34735,46,2,9,155,70,92.0,1.2,1.2,1.0,1.0,139.0,89.0,98.0,168.0,181.0,46.0,86.0,13.8,1.0,0.5,26.0,24.0,50.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
1,2023,4105118,11,1,17,160,55,86.0,0.9,9.9,1.0,2.0,123.0,52.0,87.0,NaN,NaN,NaN,NaN,15.5,1.0,1.3,22.0,11.0,31.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
2,2023,362482,36,2,13,150,65,96.0,1.0,0.8,1.0,1.0,109.0,69.0,114.0,220.0,307.0,52.0,107.0,13.5,1.0,1.1,22.0,29.0,24.0,1.0,1.0,0,NaN,NaN,NaN,NaN,NaN
3,2023,653166,11,1,13,160,70,85.0,1.0,1.2,1.0,1.0,116.0,64.0,98.0,91.0,171.0,45.0,11.0,15.6,1.0,0.9,22.0,21.0,27.0,3.0,1.0,1,0.0,NaN,NaN,NaN,0.0
4,2023,4152237,41,1,12,165,65,84.5,1.0,1.2,1.0,1.0,137.0,86.0,105.0,NaN,NaN,NaN,NaN,14.9,1.0,0.8,23.0,33.0,49.0,3.0,1.0,0,NaN,NaN,NaN,NaN,NaN


In [6]:
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 33 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   기준년도           1000000 non-null  int64  
 1   가입자일련번호        1000000 non-null  int64  
 2   시도코드           1000000 non-null  int64  
 3   성별코드           1000000 non-null  int64  
 4   연령대코드(5세단위)    1000000 non-null  int64  
 5   신장(5cm단위)      1000000 non-null  int64  
 6   체중(5kg단위)      1000000 non-null  int64  
 7   허리둘레           999589 non-null   float64
 8   시력(좌)          999816 non-null   float64
 9   시력(우)          999823 non-null   float64
 10  청력(좌)          999863 non-null   float64
 11  청력(우)          999862 non-null   float64
 12  수축기혈압          994253 non-null   float64
 13  이완기혈압          994253 non-null   float64
 14  식전혈당(공복혈당)     994186 non-null   float64
 15  총콜레스테롤         338606 non-null   float64
 16  트리글리세라이드       338606 non-null   float64
 17  HDL콜레스테롤 

In [7]:
df.isnull().sum()

기준년도                   0
가입자일련번호                0
시도코드                   0
성별코드                   0
연령대코드(5세단위)            0
신장(5cm단위)              0
체중(5kg단위)              0
허리둘레                 411
시력(좌)                184
시력(우)                177
청력(좌)                137
청력(우)                138
수축기혈압               5747
이완기혈압               5747
식전혈당(공복혈당)          5814
총콜레스테롤            661394
트리글리세라이드          661394
HDL콜레스테롤          661394
LDL콜레스테롤          667247
혈색소                 5817
요단백                11156
혈청크레아티닌             5814
혈청지오티(AST)          5816
혈청지피티(ALT)          5816
감마지티피               5813
흡연상태                  89
음주여부                  56
구강검진수검여부               0
치아우식증유무           653152
결손치 유무           1000000
치아마모증유무          1000000
제3대구치(사랑니) 이상    1000000
치석                653152
dtype: int64

### 필요없는 변수 제거

    1) 기준년도 -> 전부 2023년이므로 제거
    2) 가입자일련번호 -> 중복치 없는지 확인 후 제거
    3) 시도코드 -> 영양제 추천과 지역은 관련 없으므로 제거
    4) 구강검진수검여부 -> 치아 관련 변수 제거
    5) 치아우식증유무 -> 치아 관련 변수 제거
    6) 결손치 유무 -> 치아 관련 변수 제거
    7) 치아마모증유무 -> 치아 관련 변수 제거
    8) 제3대구치(사랑니) 이상 -> 치아 관련 변수 제거
    9) 치석 -> 치아 관련 변수 제거

In [8]:
df['가입자일련번호'].nunique()

1000000

In [9]:
col_remove = [
    '기준년도',
    '가입자일련번호',
    '시도코드',
    '구강검진수검여부',
    '치아우식증유무',
    '결손치 유무',
    '치아마모증유무',
    '제3대구치(사랑니) 이상',
    '치석',
    '총콜레스테롤',
    '트리글리세라이드',
    'HDL콜레스테롤',
    'LDL콜레스테롤'
]

df = df.drop(columns=col_remove, errors='ignore')
df.head()

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
0,2,9,155,70,92.0,1.2,1.2,1.0,1.0,139.0,89.0,98.0,13.8,1.0,0.5,26.0,24.0,50.0,1.0,1.0
1,1,17,160,55,86.0,0.9,9.9,1.0,2.0,123.0,52.0,87.0,15.5,1.0,1.3,22.0,11.0,31.0,1.0,1.0
2,2,13,150,65,96.0,1.0,0.8,1.0,1.0,109.0,69.0,114.0,13.5,1.0,1.1,22.0,29.0,24.0,1.0,1.0
3,1,13,160,70,85.0,1.0,1.2,1.0,1.0,116.0,64.0,98.0,15.6,1.0,0.9,22.0,21.0,27.0,3.0,1.0
4,1,12,165,65,84.5,1.0,1.2,1.0,1.0,137.0,86.0,105.0,14.9,1.0,0.8,23.0,33.0,49.0,3.0,1.0


In [10]:
# 20대에 해당하는 연령대코드(5세 단위)만 남기기
df = df[df['연령대코드(5세단위)'].isin([5, 6])]

In [11]:
df.isnull().sum()

성별코드             0
연령대코드(5세단위)      0
신장(5cm단위)        0
체중(5kg단위)        0
허리둘레            56
시력(좌)           19
시력(우)           19
청력(좌)            7
청력(우)            7
수축기혈압            2
이완기혈압            2
식전혈당(공복혈당)       4
혈색소              4
요단백            525
혈청크레아티닌          4
혈청지오티(AST)       4
혈청지피티(ALT)       4
감마지티피            4
흡연상태             7
음주여부             1
dtype: int64

## 결측 확인

In [12]:
# 시력(좌)와 시력(우)의 결측치가 같은 행 확인
nan_eyes = df[df['시력(좌)'].isnull() & df['시력(우)'].isnull()]

# 결과 출력
if not nan_eyes.empty:
    print("시력(좌)와 시력(우)에서 결측치가 같은 행:")
    display(nan_eyes)
else:
    print("시력(좌)와 시력(우)에서 결측치가 같은 행은 없습니다.")

시력(좌)와 시력(우)에서 결측치가 같은 행:


,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
41891,1,6,165,55,74.0,NaN,NaN,NaN,NaN,102.0,66.0,93.0,14.6,1.0,0.9,20.0,15.0,14.0,1.0,0.0
50847,1,5,170,75,87.0,NaN,NaN,3.0,3.0,118.0,70.0,236.0,14.9,1.0,0.6,13.0,19.0,33.0,1.0,0.0
144925,1,5,190,85,99.5,NaN,NaN,NaN,NaN,136.0,74.0,169.0,15.6,3.0,0.9,103.0,28.0,49.0,1.0,0.0
159801,1,6,165,45,78.0,NaN,NaN,1.0,1.0,130.0,80.0,95.0,16.5,1.0,0.7,31.0,20.0,14.0,1.0,0.0
174182,2,5,160,50,75.0,NaN,NaN,3.0,3.0,98.0,59.0,84.0,13.1,1.0,0.7,21.0,9.0,14.0,1.0,0.0
192947,1,6,170,50,74.0,NaN,NaN,1.0,1.0,126.0,80.0,110.0,14.2,1.0,0.6,23.0,21.0,20.0,1.0,0.0
239026,1,6,165,70,86.0,NaN,NaN,1.0,1.0,118.0,78.0,85.0,13.6,1.0,0.9,40.0,22.0,23.0,1.0,0.0
415200,1,5,165,85,103.0,NaN,NaN,1.0,1.0,153.0,79.0,94.0,14.8,1.0,0.7,75.0,147.0,211.0,1.0,0.0
417703,2,5,150,40,61.0,NaN,NaN,1.0,1.0,116.0,70.0,95.0,14.0,1.0,0.5,21.0,18.0,17.0,1.0,0.0
460312,1,6,165,60,75.0,NaN,NaN,3.0,3.0,113.0,55.0,87.0,16.8,1.0,1.1,18.0,23.0,17.0,1.0,0.0


In [13]:
# 청력(좌)와 청력(우)의 결측치가 같은 행 확인
nan_ears = df[df['청력(좌)'].isnull() & df['청력(우)'].isnull()]

# 결과 출력
if not nan_ears.empty:
    print("청력(좌)와 청력(우)에서 결측치가 같은 행:")
    display(nan_ears)
else:
    print("청력(좌)와 청력(우)에서 결측치가 같은 행은 없습니다.")

청력(좌)와 청력(우)에서 결측치가 같은 행:


,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
41891,1,6,165,55,74.0,NaN,NaN,NaN,NaN,102.0,66.0,93.0,14.6,1.0,0.9,20.0,15.0,14.0,1.0,0.0
85798,2,6,140,55,81.0,0.7,0.9,NaN,NaN,95.0,48.0,98.0,14.3,1.0,0.7,18.0,17.0,13.0,1.0,0.0
144925,1,5,190,85,99.5,NaN,NaN,NaN,NaN,136.0,74.0,169.0,15.6,3.0,0.9,103.0,28.0,49.0,1.0,0.0
949986,2,6,165,80,91.0,NaN,NaN,NaN,NaN,132.0,79.0,80.0,12.7,NaN,0.6,33.0,40.0,83.0,2.0,1.0
951429,1,5,130,35,65.0,NaN,NaN,NaN,NaN,108.0,70.0,74.0,15.6,1.0,0.6,21.0,21.0,23.0,1.0,0.0
972753,1,6,180,60,85.0,NaN,NaN,NaN,NaN,117.0,68.0,93.0,15.7,1.0,0.6,19.0,30.0,45.0,1.0,0.0
984772,2,6,170,55,69.0,NaN,NaN,NaN,NaN,109.0,66.0,92.0,14.0,NaN,0.7,17.0,10.0,19.0,1.0,0.0


In [14]:
# 수축기혈압과 이완기혈압의 결측치가 같은 행 확인
nan_bloodpressure = df[df['수축기혈압'].isnull() & df['이완기혈압'].isnull()]

# 결과 출력
if not nan_bloodpressure.empty:
    print("수축기혈압과 이완기혈압에서 결측치가 같은 행:")
    display(nan_bloodpressure)
else:
    display("수축기혈압과 이완기혈압에서 결측치가 같은 행은 없습니다.")

수축기혈압과 이완기혈압에서 결측치가 같은 행:


,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
295400,2,6,170,50,60.0,0.8,0.9,1.0,1.0,NaN,NaN,69.0,13.8,3.0,0.6,15.0,12.0,16.0,1.0,1.0
555590,1,5,175,60,70.0,1.0,1.0,1.0,1.0,NaN,NaN,90.0,15.6,1.0,0.9,25.0,17.0,16.0,1.0,1.0


In [15]:
nan_blood = df[df['혈청크레아티닌'].isnull() & df['혈청지오티(AST)'].isnull()]

# 결과 출력
if not nan_blood.empty:
    print("식전혈당(공복혈당), 혈색소, 혈청크레아티닌, 혈청지오티(AST), 혈청지피티(ALT), 감마지티피에서 결측치가 같은 행:")
    display(nan_blood)
else:
    display("식전혈당(공복혈당), 혈색소, 혈청크레아티닌, 혈청지오티(AST), 혈청지피티(ALT), 감마지티피에서 결측치가 같은 행은 없습니다.")

식전혈당(공복혈당), 혈색소, 혈청크레아티닌, 혈청지오티(AST), 혈청지피티(ALT), 감마지티피에서 결측치가 같은 행:


,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부
270468,1,6,175,80,95.5,0.1,0.1,1.0,1.0,140.0,97.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,0.0
321341,1,5,170,95,103.0,0.9,0.9,1.0,1.0,130.0,88.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,0.0
462769,1,6,160,70,98.1,0.5,0.5,1.0,1.0,130.0,87.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,0.0
990448,1,5,165,60,82.0,9.9,9.9,3.0,3.0,109.0,79.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,0.0


In [16]:
# 각 행의 모든 컬럼이 동일한 행을 출력함 => 동일행이 없음을 확인
samerow = df[df.nunique(axis=1) == 1]
samerow

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부


### 결측 처리 하지 않고 진행

### '시력' 변수 생성

In [17]:
df['시력(좌)'].isnull().sum()

np.int64(19)

In [18]:
df['시력(우)'].isnull().sum()

np.int64(19)

In [19]:
# 두 변수의 차이 계산
df['차이'] = abs(df['시력(좌)'] - df['시력(우)'])

# '시력' 변수 생성
df['시력'] = df.apply(
    lambda row: (
        row['시력(좌)'] if pd.isnull(row['시력(우)']) else
        row['시력(우)'] if pd.isnull(row['시력(좌)']) else
        (row['시력(좌)'] + row['시력(우)']) / 2 if row['차이'] < 0.5 else
        min(row['시력(좌)'], row['시력(우)'])
    ),
    axis=1
)

In [20]:
print("시력 변수 생성:")
display(df[['시력(좌)', '시력(우)', '차이', '시력']])

outliers_df = df[df['차이'] >= 0.5]
print("차이가 0.5 이상인 데이터:")
display(outliers_df[['시력(좌)', '시력(우)', '차이', '시력']])

시력 변수 생성:


,시력(좌),시력(우),차이,시력
5,1.0,1.2,0.2,1.10
7,0.9,0.8,0.1,0.85
11,1.2,1.2,0.0,1.20
22,1.0,1.0,0.0,1.00
33,1.2,1.5,0.3,1.35
...,...,...,...,...
999949,1.0,1.2,0.2,1.10
999964,1.2,1.2,0.0,1.20
999977,1.2,1.0,0.2,1.10
999978,1.2,1.2,0.0,1.20


차이가 0.5 이상인 데이터:


,시력(좌),시력(우),차이,시력
92,1.0,0.4,0.6,0.4
156,0.5,1.0,0.5,0.5
176,1.2,2.0,0.8,1.2
227,0.4,0.9,0.5,0.4
329,1.0,1.5,0.5,1.0
...,...,...,...,...
998865,0.5,1.2,0.7,0.5
999106,1.0,0.5,0.5,0.5
999606,1.5,2.0,0.5,1.5
999834,0.9,0.2,0.7,0.2


In [21]:
df = df.drop(columns=['시력(좌)', '시력(우)', '차이'], errors='ignore')
df.head()

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,청력(좌),청력(우),수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력
5,2,5,170,50,69.2,1.0,1.0,92.0,60.0,93.0,12.2,1.0,0.6,16.0,11.0,12.0,1.0,1.0,1.10
7,1,6,170,75,91.0,1.0,1.0,120.0,69.0,92.0,14.9,1.0,0.7,31.0,48.0,16.0,1.0,1.0,0.85
11,2,6,155,55,74.4,1.0,1.0,109.0,67.0,90.0,13.7,1.0,0.6,12.0,8.0,12.0,1.0,1.0,1.20
22,2,5,165,70,79.0,1.0,1.0,152.0,91.0,114.0,13.7,1.0,0.7,21.0,25.0,19.0,1.0,1.0,1.00
33,1,5,175,75,76.0,1.0,1.0,110.0,70.0,96.0,14.9,1.0,1.2,28.0,21.0,24.0,3.0,1.0,1.35


In [22]:
df['시력'].isnull().sum() # 결측 유지 확인

np.int64(19)

### '청력' 변수 생성

In [23]:
print(df['청력(좌)'].isnull().sum())
df['청력(우)'].isnull().sum()

7


np.int64(7)

In [24]:
# '청력(좌)'와 '청력(우)'의 3.0 값을 NaN으로 변경
df['청력(좌)'] = df['청력(좌)'].replace(3.0, np.nan)
df['청력(우)'] = df['청력(우)'].replace(3.0, np.nan)

In [25]:
# '청력' 변수 생성
df['청력'] = df.apply(
    lambda row: (
        row['청력(좌)'] if pd.isnull(row['청력(우)']) else
        row['청력(우)'] if pd.isnull(row['청력(좌)']) else
        min(row['청력(좌)'], row['청력(우)']) if row['청력(좌)'] != row['청력(우)'] else
        row['청력(좌)']
    ),
    axis=1
)

# 결과 확인
df[['청력(좌)', '청력(우)', '청력']]

,청력(좌),청력(우),청력
5,1.0,1.0,1.0
7,1.0,1.0,1.0
11,1.0,1.0,1.0
22,1.0,1.0,1.0
33,1.0,1.0,1.0
...,...,...,...
999949,1.0,1.0,1.0
999964,1.0,1.0,1.0
999977,1.0,1.0,1.0
999978,1.0,1.0,1.0


In [26]:
df['청력(좌)'].unique()

array([ 1., nan,  2.])

In [27]:
df = df.drop(columns=['청력(좌)', '청력(우)'], errors='ignore')
df.head()

,성별코드,연령대코드(5세단위),신장(5cm단위),체중(5kg단위),허리둘레,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력
5,2,5,170,50,69.2,92.0,60.0,93.0,12.2,1.0,0.6,16.0,11.0,12.0,1.0,1.0,1.10,1.0
7,1,6,170,75,91.0,120.0,69.0,92.0,14.9,1.0,0.7,31.0,48.0,16.0,1.0,1.0,0.85,1.0
11,2,6,155,55,74.4,109.0,67.0,90.0,13.7,1.0,0.6,12.0,8.0,12.0,1.0,1.0,1.20,1.0
22,2,5,165,70,79.0,152.0,91.0,114.0,13.7,1.0,0.7,21.0,25.0,19.0,1.0,1.0,1.00,1.0
33,1,5,175,75,76.0,110.0,70.0,96.0,14.9,1.0,1.2,28.0,21.0,24.0,3.0,1.0,1.35,1.0


In [28]:
df['청력'].isnull().sum() # 결측 유지 확인(기존 7개 + 3.0값 45개)

np.int64(52)

## BMI 변수 생성
- BMI는 전체적인 비만도 지표
- 허리둘레는 내장지방 특화

In [29]:
# BMI 계산 함수
def calculate_bmi(row):
    weight = row['체중(5kg단위)']
    height = row['신장(5cm단위)']
    
    # 결측값 처리: 체중 또는 신장이 결측이면 BMI도 결측값 반환
    if pd.isnull(weight) or pd.isnull(height):
        return np.nan
    
    return weight / ((height / 100) ** 2)

df['BMI'] = df.apply(calculate_bmi, axis=1)
print(df[['체중(5kg단위)', '신장(5cm단위)', 'BMI']].head())

    체중(5kg단위)  신장(5cm단위)        BMI
5          50        170  17.301038
7          75        170  25.951557
11         55        155  22.892820
22         70        165  25.711662
33         75        175  24.489796


In [30]:
df['BMI'].isnull().sum() # 신장, 체중 변수 결측값 존재하지 X

np.int64(0)

## WHtR(Waist-to-Height Ratio) 변수 생성

In [31]:
# WHtR 계산 함수
def calculate_whtr(row):
    waist = row['허리둘레']
    height = row['신장(5cm단위)']
    
    # 결측값 처리: 허리둘레 또는 신장이 결측이면 WHtR도 결측값 반환
    if pd.isnull(waist) or pd.isnull(height):
        return np.nan
    
    return waist / height

df['WHtR'] = df.apply(calculate_whtr, axis=1)

print(df['WHtR'].describe())
print(df)

count    104763.000000
mean          0.464112
std           0.063509
min           0.300000
25%           0.418750
50%           0.454545
75%           0.500000
max           0.893333
Name: WHtR, dtype: float64
        성별코드  연령대코드(5세단위)  신장(5cm단위)  체중(5kg단위)  허리둘레  수축기혈압  이완기혈압  \
5          2            5        170         50  69.2   92.0   60.0   
7          1            6        170         75  91.0  120.0   69.0   
11         2            6        155         55  74.4  109.0   67.0   
22         2            5        165         70  79.0  152.0   91.0   
33         1            5        175         75  76.0  110.0   70.0   
...      ...          ...        ...        ...   ...    ...    ...   
999949     2            5        165         70  74.0  126.0   79.0   
999964     1            6        175         75  85.0  121.0   68.0   
999977     2            6        160         50  62.0  100.0   61.0   
999978     2            6        160         50  65.0   98.0   67.0   
999993  

In [32]:
df['WHtR'].isnull().sum() # 허리둘레 변수에 56개의 결측 존재

np.int64(56)

## 위험도 변수
### 1. BMI_WC 위험도 변수 생성
- 0 : 낮음(저체중)
- 1 : 보통(정상)
- 2 : 약간 높음(비만전단계)
- 3 : 높음(1단계 비만)
- 4 : 매우 높음(2단계 비만)
- 5 : 가장 높음(3단계 비만)

In [33]:
# 위험도 분류 함수
def categorize_risk1(row):
    gender = row['성별코드']
    bmi = row['BMI']
    waist = row['허리둘레']
    
    # 결측값 처리: 성별, BMI, 허리둘레 중 하나라도 결측이면 결측값 반환
    if pd.isnull(gender) or pd.isnull(bmi) or pd.isnull(waist):
        return np.nan
    
    low_risk_waist = 90 if gender == 1 else 85
    
    if bmi < 18.5:
        BMI_WC_risk = 0 if waist < low_risk_waist else 1
    elif 18.5 <= bmi < 23:
        BMI_WC_risk = 1 if waist < low_risk_waist else 2
    elif 23 <= bmi < 25:
        BMI_WC_risk = 2 if waist < low_risk_waist else 3
    elif 25 <= bmi < 30:
        BMI_WC_risk = 3 if waist < low_risk_waist else 4
    elif 30 <= bmi < 35:
        BMI_WC_risk = 4 if waist < low_risk_waist else 5
    else:
        BMI_WC_risk = 5
    
    return BMI_WC_risk

df['BMI_WC_risk'] = df.apply(categorize_risk1, axis=1)
print(df[['BMI', 'BMI_WC_risk']].head())

          BMI  BMI_WC_risk
5   17.301038          0.0
7   25.951557          4.0
11  22.892820          1.0
22  25.711662          3.0
33  24.489796          2.0


### 2. WHtR_risk 위험도 변수 생성
- 0 : 낮음(저체중)
- 1 : 보통(정상)
- 2 : 약간 높음(과체중)
- 3 : 높음(비만)

In [34]:
# WHtR 위험도 분류 함수
def categorize_risk2(row):
    whtr = row['WHtR']  
    
    # 결측값 처리: WHtR이 결측이면 위험도도 결측값 반환
    if pd.isnull(whtr):
        return np.nan

    if whtr < 0.43:
        WHtR_risk = 0  # 저체중
    elif 0.43 <= whtr < 0.53:
        WHtR_risk = 1  # 정상
    elif 0.53 <= whtr < 0.58:
        WHtR_risk = 2  # 과체중
    else:
        WHtR_risk = 3  # 비만
    
    return WHtR_risk

df['WHtR_risk'] = df.apply(categorize_risk2, axis=1)
print(df[['WHtR', 'WHtR_risk']].head())

        WHtR  WHtR_risk
5   0.407059        0.0
7   0.535294        2.0
11  0.480000        1.0
22  0.478788        1.0
33  0.434286        1.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104819 entries, 5 to 999993
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   성별코드         104819 non-null  int64  
 1   연령대코드(5세단위)  104819 non-null  int64  
 2   신장(5cm단위)    104819 non-null  int64  
 3   체중(5kg단위)    104819 non-null  int64  
 4   허리둘레         104763 non-null  float64
 5   수축기혈압        104817 non-null  float64
 6   이완기혈압        104817 non-null  float64
 7   식전혈당(공복혈당)   104815 non-null  float64
 8   혈색소          104815 non-null  float64
 9   요단백          104294 non-null  float64
 10  혈청크레아티닌      104815 non-null  float64
 11  혈청지오티(AST)   104815 non-null  float64
 12  혈청지피티(ALT)   104815 non-null  float64
 13  감마지티피        104815 non-null  float64
 14  흡연상태         104812 non-null  float64
 15  음주여부         104818 non-null  float64
 16  시력           104800 non-null  float64
 17  청력           104767 non-null  float64
 18  BMI          104819 non-null 

In [36]:
# 필요없는 열 제거
missing_cols_1 = ['BMI','WHtR','연령대코드(5세단위)','신장(5cm단위)','체중(5kg단위)','허리둘레'] 
df = df.drop(missing_cols_1, axis = "columns")

In [37]:
df.isnull().sum()

성별코드             0
수축기혈압            2
이완기혈압            2
식전혈당(공복혈당)       4
혈색소              4
요단백            525
혈청크레아티닌          4
혈청지오티(AST)       4
혈청지피티(ALT)       4
감마지티피            4
흡연상태             7
음주여부             1
시력              19
청력              52
BMI_WC_risk     56
WHtR_risk       56
dtype: int64

## 가경이행, 철수행 생성

- 기준 : 질병관리청 국가건강정보포털, 서울아산병원
- 아래 변수에 계산되어 포함되어 있는 변수(신장, 체중, 허리둘레 등)는 제외한 데이터셋 df 사용
- 건강한가경이행과 건강한철수행을 기준으로 잡아 각 변수별 데이터마다 유사도를 책정하여 진행할 예정
- '완벽행' 열 생성 >> 기준이 되는 가경이행, 철수행과 나머지 데이터를 구분


|**변수**|**건강한가경이행**|**건강한철수행**|
|------|---|---|
|**성별코드**|2|1|
|**수축기혈압**|105|105|
|**이완기혈압**|70|70|
|**식전혈당(공복혈당)**|90|90|
|**혈색소**|12.75|14.75|
|**요단백**|1|1|
|**혈청크레아티닌**|1.25|1.25|
|**혈청지오티(AST)**|20|20|
|**혈청지피티(ALT)**|17.5|17.5|
|**감마지티피**|21.5|37|
|**흡연상태**|1|1|
|**음주여부**|0|0|
|**시력**|0.6|0.6|
|**청력**|1|1|
|**BMI_WC_risk**|1|1|
|**WHtR_risk**|1|1|

In [38]:
#######################################가경이행 철수행 생성
gg = {
    '성별코드': 2,
    '수축기혈압': 105,
    '이완기혈압': 70.0,
    '식전혈당(공복혈당)': 90.0,
    '혈색소': 13.75,
    '요단백': 1.0,
    '혈청크레아티닌': 1.25,
    '혈청지오티(AST)': 20.0,
    '혈청지피티(ALT)': 17.5,
    '감마지티피': 21.5,
    '흡연상태': 1,  
    '음주여부': 0,  
    '시력': 0.6,
    '청력': 1.0,
    'BMI_WC_risk': 0,
    'WHtR_risk': 1
}
cs = {
    '성별코드': 1,
    '수축기혈압': 105,
    '이완기혈압': 70.0,
    '식전혈당(공복혈당)': 90.0,
    '혈색소': 14.75,
    '요단백': 1.0,
    '혈청크레아티닌': 1.25,
    '혈청지오티(AST)': 20.0,
    '혈청지피티(ALT)': 17.5,
    '감마지티피': 37,
    '흡연상태': 1,  
    '음주여부': 0,  
    '시력': 0.6,
    '청력': 1.0,
    'BMI_WC_risk': 0,
    'WHtR_risk': 1
}
healthy= pd.DataFrame([gg, cs])

df = pd.concat([df, healthy], ignore_index=True)
df.tail()

,성별코드,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
104816,2,100.0,61.0,93.0,13.60,1.0,0.80,20.0,14.0,12.0,1.0,1.0,1.1,1.0,1.0,0.0
104817,2,98.0,67.0,96.0,13.20,3.0,1.00,21.0,14.0,25.0,1.0,0.0,1.2,1.0,1.0,0.0
104818,1,125.0,80.0,92.0,16.90,1.0,1.10,30.0,20.0,16.0,1.0,1.0,0.8,1.0,2.0,0.0
104819,2,105.0,70.0,90.0,13.75,1.0,1.25,20.0,17.5,21.5,1.0,0.0,0.6,1.0,0.0,1.0
104820,1,105.0,70.0,90.0,14.75,1.0,1.25,20.0,17.5,37.0,1.0,0.0,0.6,1.0,0.0,1.0


### 성별을 기준으로 데이터셋 분리

In [39]:
male_group = df[df['성별코드'] == 1]
female_group = df[df['성별코드'] == 2]

# 나눴으므로 성별코드 제거
male_group = male_group.drop(columns=['성별코드'])
female_group = female_group.drop(columns=['성별코드'])

In [40]:
male_group

,수축기혈압,이완기혈압,식전혈당(공복혈당),혈색소,요단백,혈청크레아티닌,혈청지오티(AST),혈청지피티(ALT),감마지티피,흡연상태,음주여부,시력,청력,BMI_WC_risk,WHtR_risk
1,120.0,69.0,92.0,14.90,1.0,0.70,31.0,48.0,16.0,1.0,1.0,0.85,1.0,4.0,2.0
4,110.0,70.0,96.0,14.90,1.0,1.20,28.0,21.0,24.0,3.0,1.0,1.35,1.0,2.0,1.0
6,135.0,86.0,99.0,15.60,1.0,1.10,37.0,45.0,59.0,3.0,1.0,0.40,1.0,3.0,1.0
8,135.0,79.0,98.0,16.20,1.0,1.00,20.0,25.0,33.0,1.0,0.0,0.25,1.0,3.0,1.0
11,96.0,62.0,83.0,15.20,1.0,0.90,22.0,17.0,17.0,3.0,1.0,0.75,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104812,112.0,69.0,85.0,14.80,1.0,0.70,23.0,37.0,22.0,2.0,1.0,1.00,1.0,2.0,1.0
104813,103.0,64.0,91.0,16.20,1.0,1.10,16.0,18.0,14.0,1.0,1.0,1.50,1.0,1.0,0.0
104815,121.0,68.0,97.0,16.00,1.0,0.80,48.0,104.0,132.0,1.0,1.0,1.20,1.0,2.0,1.0
104818,125.0,80.0,92.0,16.90,1.0,1.10,30.0,20.0,16.0,1.0,1.0,0.80,1.0,2.0,0.0


In [41]:
# 데이터프레임으로 변환
#enc_male = pd.DataFrame(scaled_male_data, columns=male_group.columns)
#enc_female = pd.DataFrame(scaled_female_data, columns=female_group.columns)

### 일반 사용자와 건강한 행 분리

In [42]:
# 건강한 행
healthy_male = male_group[-1:]
healthy_female = female_group[-1:]
# 일반 사용자 행
male = male_group[:51095]
female = female_group[:53724]

## 유클리디안 유사도 계산 
- 유클리디안 거리의 역수
- 유클리디안 거리가 0에 가까울수록 유사도는 1에 가까워짐!
- 남성 그룹의 유사도 변수 : uc_similarity_male 
- 여성 그룹의 유사도 변수 : uc_similarity_female 

### 아래 코드의 알고리즘 설명
1. 빈 df 생성 ( 각 변수에 대해 유클리디안 거리 + 유사도 저장 용도 )
2. 각 변수별 유클리디안 거리 및 유사도 계산 (반복문)
3. 기준값 설정 (각 변수에 대해 비교할 기준이 되는 값을 정의)
4. 결측값 처리 ( 기준값이나 해당 변수에 결측값이 있는 경우에는, 결측값을 제외한 유효한 데이터(valid_data)만 사용하여 계산을 진행함! )
5. 유클리디안 거리 계산 ( 유효한 데이터와 기준값 간 )
6. 유클리디안 거리의 평균값으로 대체 ( 유효한 데이터의 유사도를 평균으로 대체 -> 유사도를 일관되게 계산하기 위함 )
7. 유클리디안 유사도 계산 후 df에 저장 ( 유클리디안 유사도 = 1 / (1 + avg_score) )

참고) 유효한 데이터가 없을 경우, 해당 변수의 유클리디안 거리와 유사도는 NaN으로 처리됨.

In [43]:
# 유클리디안 거리 및 유사도 계산 함수
def calculate_euclidean_distance_and_similarity(male, healthy_male, column):
    reference_value = np.array(healthy_male[column]).reshape(1, -1)
    
    # 결측값 처리: 기준값이나 해당 변수에 결측값이 있는 경우, 결측값을 제외한 유효한 데이터만 사용
    if pd.isnull(reference_value).any() or male[column].isnull().any():
        valid_data = male[column].dropna()  # 결측값 제외한 데이터
        if len(valid_data) > 0:
            # 유클리디안 거리 계산
            valid_data_array = valid_data.values.reshape(-1, 1)  # 유효한 데이터만을 2D 배열로 변환
            euclidean_scores = euclidean_distances(valid_data_array, reference_value).flatten()

            # 유사도 계산 (1 / (1 + 거리))
            euclidean_similarity = 1 / (1 + euclidean_scores)
            return euclidean_scores, euclidean_similarity, valid_data.index  # 거리, 유사도, 인덱스 반환
        else:
            # 유효한 데이터가 없으면 NaN 처리
            return np.nan, np.nan, valid_data.index
    else:
        # 기준값이 결측값이 아니고 변수에 결측값이 없을 경우, 전체 데이터를 사용하여 거리 계산
        euclidean_scores = euclidean_distances(male[[column]], reference_value).flatten()
        euclidean_similarity = 1 / (1 + euclidean_scores)  # 유사도 계산 (거리의 역수)
        return euclidean_scores, euclidean_similarity, male.index  # 전체 데이터 인덱스 반환

# 변수별 유클리디안 거리와 유사도를 저장할 빈 데이터프레임 생성
uc_similarity_male = pd.DataFrame(index=male.index)

# 각 변수별 유클리디안 거리 및 유사도 계산
for column in male.columns:
    euclidean_scores, euclidean_similarity, valid_indices = calculate_euclidean_distance_and_similarity(male, healthy_male, column)
    
    # 결과를 DataFrame에 채우기
    if isinstance(euclidean_scores, np.ndarray):  # 유효한 데이터가 존재할 때
        # valid_indices가 male의 인덱스와 일치하도록 처리
        uc_similarity_male.loc[valid_indices, column + '_euclidean_distance'] = euclidean_scores  # 유클리디안 거리 채우기
        uc_similarity_male.loc[valid_indices, column + '_similarity'] = euclidean_similarity  # 유사도 채우기
    else:  # 유효한 데이터가 없을 때 NaN 처리
        uc_similarity_male[column + '_euclidean_distance'] = np.nan
        uc_similarity_male[column + '_similarity'] = np.nan

uc_similarity_male

,수축기혈압_euclidean_distance,수축기혈압_similarity,이완기혈압_euclidean_distance,이완기혈압_similarity,식전혈당(공복혈당)_euclidean_distance,식전혈당(공복혈당)_similarity,혈색소_euclidean_distance,혈색소_similarity,요단백_euclidean_distance,요단백_similarity,혈청크레아티닌_euclidean_distance,혈청크레아티닌_similarity,혈청지오티(AST)_euclidean_distance,혈청지오티(AST)_similarity,혈청지피티(ALT)_euclidean_distance,혈청지피티(ALT)_similarity,감마지티피_euclidean_distance,감마지티피_similarity,흡연상태_euclidean_distance,흡연상태_similarity,음주여부_euclidean_distance,음주여부_similarity,시력_euclidean_distance,시력_similarity,청력_euclidean_distance,청력_similarity,BMI_WC_risk_euclidean_distance,BMI_WC_risk_similarity,WHtR_risk_euclidean_distance,WHtR_risk_similarity
1,15.0,0.062500,1.0,0.500000,2.0,0.333333,0.15,0.869565,0.0,1.0,0.55,0.645161,11.0,0.083333,30.5,0.031746,21.0,0.045455,0.0,1.000000,1.0,0.5,0.25,0.800000,0.0,1.0,4.0,0.200000,1.0,0.5
4,5.0,0.166667,0.0,1.000000,6.0,0.142857,0.15,0.869565,0.0,1.0,0.05,0.952381,8.0,0.111111,3.5,0.222222,13.0,0.071429,2.0,0.333333,1.0,0.5,0.75,0.571429,0.0,1.0,2.0,0.333333,0.0,1.0
6,30.0,0.032258,16.0,0.058824,9.0,0.100000,0.85,0.540541,0.0,1.0,0.15,0.869565,17.0,0.055556,27.5,0.035088,22.0,0.043478,2.0,0.333333,1.0,0.5,0.20,0.833333,0.0,1.0,3.0,0.250000,0.0,1.0
8,30.0,0.032258,9.0,0.100000,8.0,0.111111,1.45,0.408163,0.0,1.0,0.25,0.800000,0.0,1.000000,7.5,0.117647,4.0,0.200000,0.0,1.000000,0.0,1.0,0.35,0.740741,0.0,1.0,3.0,0.250000,0.0,1.0
11,9.0,0.100000,8.0,0.111111,7.0,0.125000,0.45,0.689655,0.0,1.0,0.35,0.740741,2.0,0.333333,0.5,0.666667,20.0,0.047619,2.0,0.333333,1.0,0.5,0.15,0.869565,0.0,1.0,0.0,1.000000,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104811,14.0,0.066667,5.0,0.166667,10.0,0.090909,1.15,0.465116,0.0,1.0,0.25,0.800000,10.0,0.090909,3.5,0.222222,11.0,0.083333,0.0,1.000000,0.0,1.0,0.90,0.526316,0.0,1.0,3.0,0.250000,0.0,1.0
104812,7.0,0.125000,1.0,0.500000,5.0,0.166667,0.05,0.952381,0.0,1.0,0.55,0.645161,3.0,0.250000,19.5,0.048780,15.0,0.062500,1.0,0.500000,1.0,0.5,0.40,0.714286,0.0,1.0,2.0,0.333333,0.0,1.0
104813,2.0,0.333333,6.0,0.142857,1.0,0.500000,1.45,0.408163,0.0,1.0,0.15,0.869565,4.0,0.200000,0.5,0.666667,23.0,0.041667,0.0,1.000000,1.0,0.5,0.90,0.526316,0.0,1.0,1.0,0.500000,1.0,0.5
104815,16.0,0.058824,2.0,0.333333,7.0,0.125000,1.25,0.444444,0.0,1.0,0.45,0.689655,28.0,0.034483,86.5,0.011429,95.0,0.010417,0.0,1.000000,1.0,0.5,0.60,0.625000,0.0,1.0,2.0,0.333333,0.0,1.0


In [44]:
uc_similarity_male.isnull().sum()

수축기혈압_euclidean_distance           1
수축기혈압_similarity                   1
이완기혈압_euclidean_distance           1
이완기혈압_similarity                   1
식전혈당(공복혈당)_euclidean_distance      4
식전혈당(공복혈당)_similarity              4
혈색소_euclidean_distance             4
혈색소_similarity                     4
요단백_euclidean_distance            46
요단백_similarity                    46
혈청크레아티닌_euclidean_distance         4
혈청크레아티닌_similarity                 4
혈청지오티(AST)_euclidean_distance      4
혈청지오티(AST)_similarity              4
혈청지피티(ALT)_euclidean_distance      4
혈청지피티(ALT)_similarity              4
감마지티피_euclidean_distance           4
감마지티피_similarity                   4
흡연상태_euclidean_distance            2
흡연상태_similarity                    2
음주여부_euclidean_distance            0
음주여부_similarity                    0
시력_euclidean_distance             11
시력_similarity                     11
청력_euclidean_distance             31
청력_similarity                     31
BMI_WC_risk_euclidean_distance     0
B

In [45]:
# 유클리디안 거리 및 유사도 계산 함수
def calculate_euclidean_distance_and_similarity(female, healthy_female, column):
    reference_value = np.array(healthy_female[column]).reshape(1, -1)
    
    # 결측값 처리: 기준값이나 해당 변수에 결측값이 있는 경우, 결측값을 제외한 유효한 데이터만 사용
    if pd.isnull(reference_value).any() or female[column].isnull().any():
        valid_data = female[column].dropna()  # 결측값 제외한 데이터
        if len(valid_data) > 0:
            # 유클리디안 거리 계산
            valid_data_array = valid_data.values.reshape(-1, 1)  # 유효한 데이터만을 2D 배열로 변환
            euclidean_scores = euclidean_distances(valid_data_array, reference_value).flatten()

            # 유사도 계산 (1 / (1 + 거리))
            euclidean_similarity = 1 / (1 + euclidean_scores)
            return euclidean_scores, euclidean_similarity, valid_data.index  # 거리, 유사도, 인덱스 반환
        else:
            # 유효한 데이터가 없으면 NaN 처리
            return np.nan, np.nan, valid_data.index
    else:
        # 기준값이 결측값이 아니고 변수에 결측값이 없을 경우, 전체 데이터를 사용하여 거리 계산
        euclidean_scores = euclidean_distances(female[[column]], reference_value).flatten()
        euclidean_similarity = 1 / (1 + euclidean_scores)  # 유사도 계산 (거리의 역수)
        return euclidean_scores, euclidean_similarity, female.index  # 전체 데이터 인덱스 반환

# 변수별 유클리디안 거리와 유사도를 저장할 빈 데이터프레임 생성
uc_similarity_female = pd.DataFrame(index=female.index)

# 각 변수별 유클리디안 거리 및 유사도 계산
for column in female.columns:
    euclidean_scores, euclidean_similarity, valid_indices = calculate_euclidean_distance_and_similarity(female, healthy_female, column)
    
    # 결과를 DataFrame에 채우기
    if isinstance(euclidean_scores, np.ndarray):  # 유효한 데이터가 존재할 때
        # valid_indices가 male의 인덱스와 일치하도록 처리
        uc_similarity_female.loc[valid_indices, column + '_euclidean_distance'] = euclidean_scores  # 유클리디안 거리 채우기
        uc_similarity_female.loc[valid_indices, column + '_similarity'] = euclidean_similarity  # 유사도 채우기
    else:  # 유효한 데이터가 없을 때 NaN 처리
        uc_similarity_female[column + '_euclidean_distance'] = np.nan
        uc_similarity_female[column + '_similarity'] = np.nan

uc_similarity_female

,수축기혈압_euclidean_distance,수축기혈압_similarity,이완기혈압_euclidean_distance,이완기혈압_similarity,식전혈당(공복혈당)_euclidean_distance,식전혈당(공복혈당)_similarity,혈색소_euclidean_distance,혈색소_similarity,요단백_euclidean_distance,요단백_similarity,혈청크레아티닌_euclidean_distance,혈청크레아티닌_similarity,혈청지오티(AST)_euclidean_distance,혈청지오티(AST)_similarity,혈청지피티(ALT)_euclidean_distance,혈청지피티(ALT)_similarity,감마지티피_euclidean_distance,감마지티피_similarity,흡연상태_euclidean_distance,흡연상태_similarity,음주여부_euclidean_distance,음주여부_similarity,시력_euclidean_distance,시력_similarity,청력_euclidean_distance,청력_similarity,BMI_WC_risk_euclidean_distance,BMI_WC_risk_similarity,WHtR_risk_euclidean_distance,WHtR_risk_similarity
0,13.0,0.071429,10.0,0.090909,3.0,0.250000,1.55,0.392157,0.0,1.000000,0.65,0.606061,4.0,0.200000,6.5,0.133333,9.5,0.095238,0.0,1.000000,1.0,0.5,0.50,0.666667,0.0,1.0,0.0,1.00,1.0,0.5
2,4.0,0.200000,3.0,0.250000,0.0,1.000000,0.05,0.952381,0.0,1.000000,0.65,0.606061,8.0,0.111111,9.5,0.095238,9.5,0.095238,0.0,1.000000,1.0,0.5,0.60,0.625000,0.0,1.0,1.0,0.50,0.0,1.0
3,47.0,0.020833,21.0,0.045455,24.0,0.040000,0.05,0.952381,0.0,1.000000,0.55,0.645161,1.0,0.500000,7.5,0.117647,2.5,0.285714,0.0,1.000000,1.0,0.5,0.40,0.714286,0.0,1.0,3.0,0.25,0.0,1.0
5,9.0,0.100000,3.0,0.250000,13.0,0.071429,4.85,0.170940,0.0,1.000000,0.65,0.606061,2.0,0.333333,5.5,0.153846,14.5,0.064516,0.0,1.000000,1.0,0.5,0.20,0.833333,0.0,1.0,1.0,0.50,1.0,0.5
7,19.0,0.050000,8.0,0.111111,1.0,0.500000,1.15,0.465116,0.0,1.000000,0.55,0.645161,3.0,0.250000,5.5,0.153846,5.5,0.153846,0.0,1.000000,0.0,1.0,0.35,0.740741,0.0,1.0,1.0,0.50,1.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104807,15.0,0.062500,0.0,1.000000,12.0,0.076923,0.25,0.800000,0.0,1.000000,0.55,0.645161,1.0,0.500000,7.5,0.117647,5.5,0.153846,0.0,1.000000,1.0,0.5,0.90,0.526316,0.0,1.0,0.0,1.00,1.0,0.5
104810,23.0,0.041667,7.0,0.125000,3.0,0.250000,0.55,0.645161,0.0,1.000000,0.25,0.800000,2.0,0.333333,8.5,0.105263,4.5,0.181818,0.0,1.000000,1.0,0.5,0.05,0.952381,0.0,1.0,1.0,0.50,0.0,1.0
104814,21.0,0.045455,9.0,0.100000,5.0,0.166667,2.55,0.281690,0.0,1.000000,0.55,0.645161,4.0,0.200000,0.5,0.666667,0.5,0.666667,2.0,0.333333,1.0,0.5,0.50,0.666667,0.0,1.0,3.0,0.25,0.0,1.0
104816,5.0,0.166667,9.0,0.100000,3.0,0.250000,0.15,0.869565,0.0,1.000000,0.45,0.689655,0.0,1.000000,3.5,0.222222,9.5,0.095238,0.0,1.000000,1.0,0.5,0.50,0.666667,0.0,1.0,1.0,0.50,1.0,0.5


In [46]:
uc_similarity_female.isnull().sum()

수축기혈압_euclidean_distance            1
수축기혈압_similarity                    1
이완기혈압_euclidean_distance            1
이완기혈압_similarity                    1
식전혈당(공복혈당)_euclidean_distance       0
식전혈당(공복혈당)_similarity               0
혈색소_euclidean_distance              0
혈색소_similarity                      0
요단백_euclidean_distance            479
요단백_similarity                    479
혈청크레아티닌_euclidean_distance          0
혈청크레아티닌_similarity                  0
혈청지오티(AST)_euclidean_distance       0
혈청지오티(AST)_similarity               0
혈청지피티(ALT)_euclidean_distance       0
혈청지피티(ALT)_similarity               0
감마지티피_euclidean_distance            0
감마지티피_similarity                    0
흡연상태_euclidean_distance             5
흡연상태_similarity                     5
음주여부_euclidean_distance             1
음주여부_similarity                     1
시력_euclidean_distance               8
시력_similarity                       8
청력_euclidean_distance              21
청력_similarity                      21
BMI_WC_risk_

## 이제 해야할 것
1. 계산한 유사도를 기반으로 그룹핑이 필요한데, 퀀타일을 기준으로 나눈 것 외에 전문의료지식을 기준으로 시도해봐야함
(어려울 거 같기는 한데, 서울아산병원이나 다른 웹사이트에서 가져와봐야 할 듯 ㅠㅠ 이건 내일 나도 찾아볼게)
2. 1.에서 정한 기준대로 그룹핑한 후에, 각 그룹별 닉네임 붙이기(이전에 9개 그룹으로 나눈 것 처럼. 시력 그룹, 청력 그룹 등등..)
3. 그 다음, 그룹핑 데이터셋을 추천시스템에 넣어서 출력해보고 잘 작동되는지 확인해볼 것(가경찬스)

아산병원 근거로 가장 보수적으로 고위험군을 만들어서, 건강행과 유사도를 계산해
-> 이제 그 유사도가 고위험군을 구분하는 그룹핑 근거가 될 수 있음. 그 유사도 보다 낮으면 고위험군으로 설정

In [47]:
# 고위험군 행 생성
ngg = {
    #'성별코드': 2,
    '수축기혈압': 140,
    '이완기혈압': 90.0,
    '식전혈당(공복혈당)': 126.0,
    '혈색소': 10.0,
    '요단백': 3,
    '혈청크레아티닌': 1.7,
    '혈청지오티(AST)': 51.0,
    '혈청지피티(ALT)': 46,
    '감마지티피': 46,
    '흡연상태': 2.5,  
    '음주여부': 1,  
    '시력': 0.0,
    '청력': 2,
    'BMI_WC_risk': 3,
    'WHtR_risk': 2.5
}

ncs = {
    #'성별코드': 1,
    '수축기혈압': 140,
    '이완기혈압': 90.0,
    '식전혈당(공복혈당)': 126.0,
    '혈색소': 12.0,
    '요단백': 3,
    '혈청크레아티닌': 1.7,
    '혈청지오티(AST)': 51,
    '혈청지피티(ALT)': 46,
    '감마지티피': 78,
    '흡연상태': 2.5,  
    '음주여부': 1,  
    '시력': 0.0,
    '청력': 2,
    'BMI_WC_risk': 3,
    'WHtR_risk': 2.5
}

# 건강한 행과 유클리디안 유사도 계산을 위해 데이터프레임으로 변환
not_healthy_male= pd.DataFrame([ngg])
not_healthy_female= pd.DataFrame([ncs])

In [48]:
def calculate_similarity(healthy, risk):
    distance = np.sqrt((healthy - risk) ** 2)  # 유클리드 거리 계산
    similarity = 1 / (1 + distance)  # 유클리디안 유사도 계산
    return similarity

# 남성과 여성의 유사도 계산
def compute_similarity(healthy_df, risk_df):
    results_df = pd.DataFrame(index=healthy_male.index)  # 결과 저장용 빈 DataFrame
    for col in healthy_df.columns:
        similarity = calculate_similarity(healthy_df[col].values[0], risk_df[col].values[0])
        results_df[f'{col}_similarity'] = [similarity]
    return results_df

# 남성 결과 계산
male_similarity = compute_similarity(healthy_male, not_healthy_male)
# 여성 결과 계산
female_similarity = compute_similarity(healthy_female, not_healthy_female)

# Step 5: 남성과 여성 결과 합치기
combined_results = pd.concat([male_similarity, female_similarity], ignore_index=True)
combined_results['Gender'] = ['Male', 'Female']  # 성별 추가

# 결과 출력
print("남성과 여성 유사도 결과:")
print(combined_results)

남성과 여성 유사도 결과:
   수축기혈압_similarity  이완기혈압_similarity  식전혈당(공복혈당)_similarity  혈색소_similarity  \
0          0.027778          0.047619               0.027027        0.173913   
1          0.027778          0.047619               0.027027        0.363636   

   요단백_similarity  혈청크레아티닌_similarity  혈청지오티(AST)_similarity  \
0        0.333333            0.689655                0.03125   
1        0.333333            0.689655                0.03125   

   혈청지피티(ALT)_similarity  감마지티피_similarity  흡연상태_similarity  음주여부_similarity  \
0               0.033898          0.100000              0.4              0.5   
1               0.033898          0.017391              0.4              0.5   

   시력_similarity  청력_similarity  BMI_WC_risk_similarity  WHtR_risk_similarity  \
0          0.625            0.5                    0.25                   0.4   
1          0.625            0.5                    0.25                   0.4   

   Gender  
0    Male  
1  Female  


위의 수치를 근거로 위험군 / 정상 분류 후 , 그룹핑 가능
- 해당 수치보다 낮거나 같으면 위험군
- 해당 수치보다 높으면 정상

In [49]:
# 일반 남성 검수자 유클리디안 유사도 추출
similarity_male = uc_similarity_male[[col for col in uc_similarity_male.columns if '_similarity' in col]]

# 일반 여성 검수자 유클리디안 유사도 추출
similarity_female = uc_similarity_female[[col for col in uc_similarity_female.columns if '_similarity' in col]]

In [50]:
similarity_male.head(5) #확인

,수축기혈압_similarity,이완기혈압_similarity,식전혈당(공복혈당)_similarity,혈색소_similarity,요단백_similarity,혈청크레아티닌_similarity,혈청지오티(AST)_similarity,혈청지피티(ALT)_similarity,감마지티피_similarity,흡연상태_similarity,음주여부_similarity,시력_similarity,청력_similarity,BMI_WC_risk_similarity,WHtR_risk_similarity
1,0.062500,0.500000,0.333333,0.869565,1.0,0.645161,0.083333,0.031746,0.045455,1.000000,0.5,0.800000,1.0,0.200000,0.5
4,0.166667,1.000000,0.142857,0.869565,1.0,0.952381,0.111111,0.222222,0.071429,0.333333,0.5,0.571429,1.0,0.333333,1.0
6,0.032258,0.058824,0.100000,0.540541,1.0,0.869565,0.055556,0.035088,0.043478,0.333333,0.5,0.833333,1.0,0.250000,1.0
8,0.032258,0.100000,0.111111,0.408163,1.0,0.800000,1.000000,0.117647,0.200000,1.000000,1.0,0.740741,1.0,0.250000,1.0
11,0.100000,0.111111,0.125000,0.689655,1.0,0.740741,0.333333,0.666667,0.047619,0.333333,0.5,0.869565,1.0,1.000000,0.5


In [51]:
similarity_female.head(5) #확인

,수축기혈압_similarity,이완기혈압_similarity,식전혈당(공복혈당)_similarity,혈색소_similarity,요단백_similarity,혈청크레아티닌_similarity,혈청지오티(AST)_similarity,혈청지피티(ALT)_similarity,감마지티피_similarity,흡연상태_similarity,음주여부_similarity,시력_similarity,청력_similarity,BMI_WC_risk_similarity,WHtR_risk_similarity
0,0.071429,0.090909,0.250000,0.392157,1.0,0.606061,0.200000,0.133333,0.095238,1.0,0.5,0.666667,1.0,1.00,0.5
2,0.200000,0.250000,1.000000,0.952381,1.0,0.606061,0.111111,0.095238,0.095238,1.0,0.5,0.625000,1.0,0.50,1.0
3,0.020833,0.045455,0.040000,0.952381,1.0,0.645161,0.500000,0.117647,0.285714,1.0,0.5,0.714286,1.0,0.25,1.0
5,0.100000,0.250000,0.071429,0.170940,1.0,0.606061,0.333333,0.153846,0.064516,1.0,0.5,0.833333,1.0,0.50,0.5
7,0.050000,0.111111,0.500000,0.465116,1.0,0.645161,0.250000,0.153846,0.153846,1.0,1.0,0.740741,1.0,0.50,0.5


### 남성 유사도 분류

In [52]:
# 기준 유사도 값 정의 (각 변수별로 위험군 기준 수치 설정)
thresholds = {
    '수축기혈압_similarity': 0.027778,
    '이완기혈압_similarity': 0.047619,
    '식전혈당(공복혈당)_similarity': 0.027027,  
    '혈색소_similarity': 0.173913,    
    '요단백_similarity': 0.333333,     
    '혈청크레아티닌_similarity': 0.689655,  
    '혈청지오티(AST)_similarity': 0.3125,     
    '혈청지피티(ALT)_similarity': 0.33898,    
    '감마지티피_similarity': 0.100000,     
    '흡연상태_similarity': 0.4,      
    '음주여부_similarity': 0.5,       
    '시력_similarity': 0.625,
    '청력_similarity': 0.5,          
    'BMI_WC_risk_similarity': 0.25,   
    'WHtR_risk_similarity': 0.4 
}

# DataFrame 생성
user_data = pd.DataFrame(similarity_male)

# 위험군 분류 함수 정의
def classify_risk(value, threshold):
    """
    주어진 값이 기준 유사도 수치보다 작거나 같으면 'risk', 아니면 'normal'로 분류, 만약 값이 NaN이면 NaN을 그대로 반환.
    """
    if pd.isna(value):  # 값이 NaN인 경우
        return np.nan
    return 'risk' if value <= threshold else 'normal'

# 사용자 데이터셋에 대해 각 변수별 분류 결과 추가
def classify_all_variables(user_data, thresholds):
    # 결과를 저장할 새로운 DataFrame
    result_df = user_data.copy()
    
    # 각 변수별로 분류
    for column in user_data.columns:
        if column in thresholds:  # 임계값이 설정된 변수에 대해서만 처리
            result_df[f'{column}_분류'] = result_df[column].apply(classify_risk, threshold=thresholds[column])
    
    return result_df

# 사용자 데이터에 위험군 분류 추가
final_result = classify_all_variables(user_data, thresholds)

# 결과 출력
print(final_result.head(5))

    수축기혈압_similarity  이완기혈압_similarity  식전혈당(공복혈당)_similarity  혈색소_similarity  \
1           0.062500          0.500000               0.333333        0.869565   
4           0.166667          1.000000               0.142857        0.869565   
6           0.032258          0.058824               0.100000        0.540541   
8           0.032258          0.100000               0.111111        0.408163   
11          0.100000          0.111111               0.125000        0.689655   

    요단백_similarity  혈청크레아티닌_similarity  혈청지오티(AST)_similarity  \
1              1.0            0.645161               0.083333   
4              1.0            0.952381               0.111111   
6              1.0            0.869565               0.055556   
8              1.0            0.800000               1.000000   
11             1.0            0.740741               0.333333   

    혈청지피티(ALT)_similarity  감마지티피_similarity  흡연상태_similarity  음주여부_similarity  \
1                0.031746          0.0454

In [53]:
classification_male = final_result[[col for col in final_result.columns if '_분류' in col]]
print(classification_male.head(5)) #분류 결과 확인

   수축기혈압_similarity_분류 이완기혈압_similarity_분류 식전혈당(공복혈당)_similarity_분류  \
1               normal              normal                   normal   
4               normal              normal                   normal   
6               normal              normal                   normal   
8               normal              normal                   normal   
11              normal              normal                   normal   

   혈색소_similarity_분류 요단백_similarity_분류 혈청크레아티닌_similarity_분류  \
1             normal            normal                  risk   
4             normal            normal                normal   
6             normal            normal                normal   
8             normal            normal                normal   
11            normal            normal                normal   

   혈청지오티(AST)_similarity_분류 혈청지피티(ALT)_similarity_분류 감마지티피_similarity_분류  \
1                      risk                     risk                risk   
4                      risk         

분류를 기반으로 각 질병 매칭

In [54]:
# 질병 분류를 저장할 데이터셋
df = pd.DataFrame(classification_male)

# 질병별 변수 그룹 정의
disease_groups = {
    'Hypertension': ['수축기혈압_similarity_분류', '이완기혈압_similarity_분류'],
    'Anemia': ['혈색소_similarity_분류'],
    'Liver/Kidney Disease': ['혈청크레아티닌_similarity_분류','혈청지오티(AST)_similarity_분류', '혈청지피티(ALT)_similarity_분류', '감마지티피_similarity_분류'],
    'Weight management': ['BMI_WC_risk_similarity_분류','WHtR_risk_similarity_분류'],
    'Smoking_Alchol': ['흡연상태_similarity_분류','음주여부_similarity_분류'],
    'Vision': ['시력_similarity_분류'],
    'Hearing': ['청력_similarity_분류'],
    'Diabetes': ['요단백_similarity_분류', '식전혈당(공복혈당)_similarity_분류', 'BMI_WC_risk_similarity_분류', 'WHtR_risk_similarity_분류'],
    'Cardiovascular': ['수축기혈압_similarity_분류', '이완기혈압_similarity_분류', '식전혈당(공복혈당)_similarity_분류', 'BMI_WC_risk_similarity_분류', 'WHtR_risk_similarity_분류', '혈청지오티(AST)_similarity_분류', '혈청지피티(ALT)_similarity_분류', '감마지티피_similarity_분류']
}

# 질병별 위험도 평가 함수
def evaluate_risk(group, row):
    for variable in group:
        if pd.isna(row[variable]):  # 변수값이 NaN이면 NaN 반환
            return np.nan
        if row[variable] == 'risk': # 변수값이 'risk'이면 'risk' 반환
            return 'risk'
    return 'normal' # 'risk'가 없으면 'normal' 반환

# 질병 그룹별 위험도 계산
for disease, variables in disease_groups.items():
    df[disease] = df.apply(lambda row: evaluate_risk(variables, row), axis=1)

# 질병 분류 결과 추출
disease_male = list(disease_groups.keys())
print(df[disease_male])

       Hypertension  Anemia Liver/Kidney Disease Weight management  \
1            normal  normal                 risk              risk   
4            normal  normal                 risk            normal   
6            normal  normal                 risk              risk   
8            normal  normal                 risk              risk   
11           normal  normal                 risk            normal   
...             ...     ...                  ...               ...   
104811       normal  normal                 risk              risk   
104812       normal  normal                 risk            normal   
104813       normal  normal                 risk            normal   
104815       normal  normal                 risk            normal   
104818       normal  normal                 risk            normal   

       Smoking_Alchol  Vision Hearing Diabetes Cardiovascular  
1                risk  normal  normal     risk           risk  
4                risk    risk  

### 여성 유사도 분류

In [55]:
# 기준 유사도 값 정의 (각 변수별로 위험군 기준 수치 설정)
thresholds = {
    '수축기혈압_similarity': 0.027778,
    '이완기혈압_similarity': 0.047619,
    '식전혈당(공복혈당)_similarity': 0.027027,  
    '혈색소_similarity': 0.363636,    
    '요단백_similarity': 0.333333,     
    '혈청크레아티닌_similarity': 0.689655,  
    '혈청지오티(AST)_similarity': 0.3125,     
    '혈청지피티(ALT)_similarity': 0.33898,    
    '감마지티피_similarity': 0.017391,     
    '흡연상태_similarity': 0.4,      
    '음주여부_similarity': 0.5,       
    '시력_similarity': 0.625,
    '청력_similarity': 0.5,          
    'BMI_WC_risk_similarity': 0.25,   
    'WHtR_risk_similarity': 0.4 
}

# DataFrame 생성
user_data_female = pd.DataFrame(similarity_female)

# 위험군 분류 함수 정의
def classify_risk(value, threshold):
    """
    주어진 값이 기준 유사도 수치보다 작거나 같으면 'risk', 아니면 'normal'로 분류, 만약 값이 NaN이면 NaN을 그대로 반환.
    """
    if pd.isna(value):  # 값이 NaN인 경우
        return np.nan
    return 'risk' if value <= threshold else 'normal'

# 사용자 데이터셋에 대해 각 변수별 분류 결과 추가
def classify_all_variables(user_data_female, thresholds):
    # 결과를 저장할 새로운 DataFrame
    result_df_female = user_data_female.copy()
    
    # 각 변수별로 분류
    for column in user_data_female.columns:
        if column in thresholds:  # 임계값이 설정된 변수에 대해서만 처리
            result_df_female[f'{column}_분류'] = result_df_female[column].apply(classify_risk, threshold=thresholds[column])
    
    return result_df_female

# 사용자 데이터에 위험군 분류 추가
final_result_female = classify_all_variables(user_data_female, thresholds)

# 결과 출력
print(final_result_female.head(5))

   수축기혈압_similarity  이완기혈압_similarity  식전혈당(공복혈당)_similarity  혈색소_similarity  \
0          0.071429          0.090909               0.250000        0.392157   
2          0.200000          0.250000               1.000000        0.952381   
3          0.020833          0.045455               0.040000        0.952381   
5          0.100000          0.250000               0.071429        0.170940   
7          0.050000          0.111111               0.500000        0.465116   

   요단백_similarity  혈청크레아티닌_similarity  혈청지오티(AST)_similarity  \
0             1.0            0.606061               0.200000   
2             1.0            0.606061               0.111111   
3             1.0            0.645161               0.500000   
5             1.0            0.606061               0.333333   
7             1.0            0.645161               0.250000   

   혈청지피티(ALT)_similarity  감마지티피_similarity  흡연상태_similarity  음주여부_similarity  \
0               0.133333          0.095238            

In [56]:
classification_female = final_result_female[[col for col in final_result_female.columns if '_분류' in col]]
print(classification_female.head(5)) #분류 결과 확인

  수축기혈압_similarity_분류 이완기혈압_similarity_분류 식전혈당(공복혈당)_similarity_분류  \
0              normal              normal                   normal   
2              normal              normal                   normal   
3                risk                risk                   normal   
5              normal              normal                   normal   
7              normal              normal                   normal   

  혈색소_similarity_분류 요단백_similarity_분류 혈청크레아티닌_similarity_분류  \
0            normal            normal                  risk   
2            normal            normal                  risk   
3            normal            normal                  risk   
5              risk            normal                  risk   
7            normal            normal                  risk   

  혈청지오티(AST)_similarity_분류 혈청지피티(ALT)_similarity_분류 감마지티피_similarity_분류  \
0                     risk                     risk              normal   
2                     risk                     ris

In [57]:
# 질병 분류를 저장할 데이터셋
df_female = pd.DataFrame(classification_female)

# 질병별 위험도 평가 함수
def evaluate_risk(group, row):
    for variable in group:
        if pd.isna(row[variable]):  # 변수값이 NaN이면 NaN 반환
            return np.nan
        if row[variable] == 'risk': # 변수값이 'risk'이면 'risk' 반환
            return 'risk'
    return 'normal' # 'risk'가 없으면 'normal' 반환

# 질병 그룹별 위험도 계산
for disease, variables in disease_groups.items():
    df_female[disease] = df_female.apply(lambda row: evaluate_risk(variables, row), axis=1)

# 질병 분류 결과 추출
disease_female = list(disease_groups.keys())
print(df_female[disease_female])

       Hypertension  Anemia Liver/Kidney Disease Weight management  \
0            normal  normal                 risk            normal   
2            normal  normal                 risk            normal   
3              risk  normal                 risk              risk   
5            normal    risk                 risk            normal   
7            normal  normal                 risk            normal   
...             ...     ...                  ...               ...   
104807       normal  normal                 risk            normal   
104810       normal  normal                 risk            normal   
104814       normal    risk                 risk              risk   
104816       normal  normal                 risk            normal   
104817       normal  normal                 risk            normal   

       Smoking_Alchol  Vision Hearing Diabetes Cardiovascular  
0                risk  normal  normal   normal           risk  
2                risk    risk  

In [58]:
# 남성 여성 데이터셋 합치기
disease = pd.concat([df[disease_male],df_female[disease_female]], axis=0)
print(disease)

       Hypertension  Anemia Liver/Kidney Disease Weight management  \
1            normal  normal                 risk              risk   
4            normal  normal                 risk            normal   
6            normal  normal                 risk              risk   
8            normal  normal                 risk              risk   
11           normal  normal                 risk            normal   
...             ...     ...                  ...               ...   
104807       normal  normal                 risk            normal   
104810       normal  normal                 risk            normal   
104814       normal    risk                 risk              risk   
104816       normal  normal                 risk            normal   
104817       normal  normal                 risk            normal   

       Smoking_Alchol  Vision Hearing Diabetes Cardiovascular  
1                risk  normal  normal     risk           risk  
4                risk    risk  

In [59]:
count = disease.apply(lambda x: x.value_counts())
print(count)

        Hypertension  Anemia  Liver/Kidney Disease  Weight management  \
normal        100468   99742                  1654              72849   
risk            4349    5073                103161              31914   

        Smoking_Alchol  Vision  Hearing  Diabetes  Cardiovascular  
normal           14894   73927   104684     71696            3013  
risk             89918   30873       83     32539          101746  


In [100]:
# CSV로 저장
#disease.to_csv('disease.csv', encoding='cp949')
#print("CSV 파일로 저장되었습니다.")

CSV 파일로 저장되었습니다.


In [70]:
disease

,Hypertension,Anemia,Liver/Kidney Disease,Weight management,Smoking_Alchol,Vision,Hearing,Diabetes,Cardiovascular,best_match
1,normal,normal,risk,risk,risk,normal,normal,risk,risk,NaN
4,normal,normal,risk,normal,risk,risk,normal,normal,risk,NaN
6,normal,normal,risk,risk,risk,normal,normal,risk,risk,NaN
8,normal,normal,risk,risk,normal,normal,normal,risk,risk,NaN
11,normal,normal,risk,normal,risk,normal,normal,normal,risk,NaN
...,...,...,...,...,...,...,...,...,...,...
104807,normal,normal,risk,normal,risk,risk,normal,normal,risk,NaN
104810,normal,normal,risk,normal,risk,normal,normal,normal,risk,NaN
104814,normal,risk,risk,risk,risk,normal,normal,risk,risk,NaN
104816,normal,normal,risk,normal,risk,normal,normal,normal,risk,NaN


In [71]:
# 각 행에서 값이 "risk"인 컬럼 이름을 저장하는 새로운 열 추가
disease['Risk_Columns'] = disease.apply(lambda row: list(row.index[row == "risk"]), axis=1)

# 결과 확인
print(disease.head())


   Hypertension  Anemia Liver/Kidney Disease Weight management Smoking_Alchol  \
1        normal  normal                 risk              risk           risk   
4        normal  normal                 risk            normal           risk   
6        normal  normal                 risk              risk           risk   
8        normal  normal                 risk              risk         normal   
11       normal  normal                 risk            normal           risk   

    Vision Hearing Diabetes Cardiovascular best_match  \
1   normal  normal     risk           risk        NaN   
4     risk  normal   normal           risk        NaN   
6   normal  normal     risk           risk        NaN   
8   normal  normal     risk           risk        NaN   
11  normal  normal   normal           risk        NaN   

                                         Risk_Columns  
1   [Liver/Kidney Disease, Weight management, Smok...  
4   [Liver/Kidney Disease, Smoking_Alchol, Vision,...  
6 

In [60]:
product = pd.read_csv("C:/Users/jeong/Desktop/baf/product.csv")

In [80]:
# 각 열에서 값이 1인 컬럼의 이름을 저장하는 새로운 컬럼 추가
product['disease_name'] = product.apply(lambda row: list(row.index[row == 1]), axis=1)

# 결과 확인
print(product.head())


                  product  Vision  Cardiovascular  Smoking_Alchol  \
0  164 루테인지아잔틴 알티지오메가3 GR     1.0             1.0             NaN   
1                 38 발효효소     NaN             NaN             NaN   
2            곡물효소 프로바이오틱스     NaN             NaN             NaN   
3    관절엔 소연골 뮤코다당단백 콘드로이친     NaN             1.0             1.0   
4          눈 건강엔 루테인 프리미엄     1.0             NaN             NaN   

   Weight management  Liver/Kidney Disease  Anemia  Diabetes  Hypertension  \
0                NaN                   1.0     NaN       NaN           1.0   
1                1.0                   NaN     NaN       NaN           NaN   
2                1.0                   NaN     NaN       NaN           NaN   
3                NaN                   1.0     NaN       NaN           1.0   
4                NaN                   NaN     NaN       NaN           NaN   

   Hearing  score                                       disease_name  
0      1.0      1  [Vision, Cardiovascular, L

In [79]:
product.head()

,product,Vision,Cardiovascular,Smoking_Alchol,Weight management,Liver/Kidney Disease,Anemia,Diabetes,Hypertension,Hearing,score
0,164 루테인지아잔틴 알티지오메가3 GR,1.0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,1.0,1
1,38 발효효소,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2
2,곡물효소 프로바이오틱스,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,145
3,관절엔 소연골 뮤코다당단백 콘드로이친,NaN,1.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,14
4,눈 건강엔 루테인 프리미엄,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,272


In [85]:
def find_most_matching_product(risk_columns):
    max_overlap = 0
    best_match = None

    # risk_columns가 리스트가 아닌 경우 처리
    if not isinstance(risk_columns, list):
        return None

    for _, product_row in product.iterrows():
        # product['disease_name']가 리스트가 아닌 경우 처리
        disease_names = product['disease_name']
        if not isinstance(disease_names, list):
            continue

        # 겹치는 값 계산
        overlap = len(set(risk_columns) & set(disease_names))

        if overlap > max_overlap:
            max_overlap = overlap
            best_match = product['product']  # 최적의 매칭 제품 이름 저장

    return best_match


In [ ]:
disease['best_match'] = disease['Risk_Columns'].apply(find_most_matching_product)